<h1>This is the code for COM3013 Coursework</h1>
<h2>Mofe Awosanya, Ebose Odijie, Hishaam Khan</h2>

These are all the imports

In [1]:
import pickle

This function will allow us to open each data batch as they are python "pickled" files. These pickled files need to be unpickled.

In [2]:
def read_data(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

Now we will run each data batch into the function.

In [3]:
data_batch_1 = read_data("cifar-10-python/cifar-10-batches-py/data_batch_1")
data_batch_2 = read_data("cifar-10-python/cifar-10-batches-py/data_batch_2")
data_batch_3 = read_data("cifar-10-python/cifar-10-batches-py/data_batch_3")
data_batch_4 = read_data("cifar-10-python/cifar-10-batches-py/data_batch_4")
data_batch_5 = read_data("cifar-10-python/cifar-10-batches-py/data_batch_5")

test_batch = read_data("cifar-10-python/cifar-10-batches-py/test_batch")

Now to test if it has been read correctly. There should be 10,000 images per batch.

In [6]:
print(len(data_batch_1)) #idk if this right

4
